# KoSimCSE training (Supervised) on SageMaker
---

## Overview 
바로 이전 모듈까지는 기존에 온프레미스에서 개발했던 환경과 동일한 환경으로 모델을 빌드하고 훈련했습니다. 하지만 아래와 같은 상황들에서도 기존 환경을 사용하는 것이 바람직할까요?

- 온프레미스의 GPU가 총 1장으로 훈련 시간이 너무 오래 소요됨
- 가용 서버 대수가 2대인데 10개의 딥러닝 모델을 동시에 훈련해야 함
- 필요한 상황에만 GPU를 활용

Amazon SageMaker는 데이터 과학자들 및 머신 러닝 엔지니어들을 위한 완전 관리형 머신 러닝 서비스로 훈련 및 추론 수행 시 인프라 설정에 대한 추가 작업이 필요하지 있기에, 단일 GPU 기반의 딥러닝 훈련을 포함한 멀티 GPU 및 멀티 인스턴스 분산 훈련을 보다 쉽고 빠르게 수행할 수 있습니다. SageMaker는 다양한 유즈케이스들에 적합한 예제들을 지속적으로 업데이트하고 있으며, 한국어 세션 및 자료들도 제공되고 있습니다.

### Note
- 이미 기본적인 Hugging Face 용법 및 자연어 처리에 익숙하신 분들은 앞 모듈을 생략하고 이 모듈부터 핸즈온을 시작하셔도 됩니다.
- 이 노트북은 SageMaker 기본 API를 참조하므로, SageMaker Studio, SageMaker 노트북 인스턴스 또는 AWS CLI가 설정된 로컬 시스템에서 실행해야 합니다. SageMaker Studio 또는 SageMaker 노트북 인스턴스를 사용하는 경우 PyTorch 기반 커널을 선택하세요.
- 훈련(Training) job 수행 시 최소 `ml.g4dn.xlarge` 훈련 인스턴스를 권장하며, 분산 훈련 수행 시에는 `ml.g5.12xlarge` 훈련 인스턴스를 권장합니다. 만약 인스턴스 사용에 제한이 걸려 있다면 [Request a service quota increase for SageMaker resources](https://docs.aws.amazon.com/sagemaker/latest/dg/regions-quotas.html#service-limit-increase-request-procedure)를 참조하여 인스턴스 제한을 해제해 주세요.

<br>

## 1. Preparation
---

In [ ]:
import time
import boto3
import sagemaker

sess = sagemaker.Session()
region = boto3.Session().region_name
bucket = None
if bucket is None and sess is not None:
    bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=bucket)

print(f"SageMaker role arn: {role}")
print(f"SageMaker bucket: {sess.default_bucket()}")
print(f"SageMaker session region: {sess.boto_region_name}")

In [ ]:
%%bash
aws configure set default.s3.max_concurrent_requests 100
aws configure set default.s3.max_queue_size 10000
aws configure set default.s3.multipart_threshold 1GB
aws configure set default.s3.multipart_chunksize 64MB

In [ ]:
bucket_prefix = "korsimcse"
model_name = "roberta-base"

dataset_dir = "dataset-sup-train"
s3_model_path = f"s3://{bucket}/{bucket_prefix}/model/{model_name}/"
s3_dataset_path = f"s3://{bucket}/{bucket_prefix}/dataset/"

In [ ]:
!aws s3 sync {dataset_dir} {s3_dataset_path}

<br>

## 2. SageMaker Training
---

SageMaker에 대한 대표적인 오해가 여전히 많은 분들이 SageMaker 훈련을 위해 소스 코드를 전면적으로 수정해야 한다고 생각합니다. 하지만, 실제로는 별도의 소스 코드 수정 없이 기존 여러분이 사용했던 파이썬 스크립트에 SageMaker 훈련에 필요한 SageMaker 전용 환경 변수들만 추가하면 됩니다.

SageMaker 훈련은 훈련 작업을 호출할 때, 1) 훈련 EC2 인스턴스 프로비저닝 - 2) 컨테이너 구동을 위한 도커 이미지 및 훈련 데이터 다운로드 - 3) 컨테이너 구동 - 4) 컨테이너 환경에서 훈련 수행 - 5) 컨테이너 환경에서 S3의 특정 버킷에 저장 - 6) 훈련 인스턴스 종료로 구성됩니다. 따라서, 훈련 수행 로직은 아래 예시와 같이 기존 개발 환경과 동일합니다.

`/opt/conda/bin/python train_hf.py --num_epochs 5 --train_batch_size 32 ...`

이 과정에서 컨테이너 환경에 필요한 환경 변수(예: 모델 경로, 훈련 데이터 경로) 들은 사전에 지정되어 있으며, 이 환경 변수들이 설정되어 있어야 훈련에 필요한 파일들의 경로를 인식할 수 있습니다. 대표적인 환경 변수들에 대한 자세한 내용은 https://github.com/aws/sagemaker-containers#important-environment-variables 을 참조하세요.

### Setup SageMaker Estimator

In [ ]:
entry_point = "sup_run.sh"

In [ ]:
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.inputs import TrainingInput
from sagemaker.pytorch import PyTorch

# Define Training Job Name 
job_name = f"kosimcse-{model_name}-supervised-{time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())}"
print(job_name)

# See https://github.com/aws/deep-learning-containers/blob/master/available_images.md
image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:2.0.1-gpu-py310-cu118-ubuntu20.04-sagemaker"
hparams = {}

instance_type = "ml.g5.12xlarge"
max_run = 6*60*60 # 6 hours
use_spot_instances = False
if use_spot_instances:
    max_wait = 12*60*60 # 12 hours: spot instance waiting + max runtime
else:
    max_wait = None
    
# Create the Estimator
estimator = PyTorch(
    image_uri=image_uri,
    entry_point=entry_point,           # train script
    source_dir="src",               # directory which includes all the files needed for training
    instance_type=instance_type, # instances type used for the training job
    instance_count=1,               # the number of instances used for training
    base_job_name=job_name,         # the name of the training job
    role=role,                      # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size=300,                # the size of the EBS volume in GB
    py_version="py310",             # the python version used in the training job
    hyperparameters=hparams,
    debugger_hook_config=False,
    disable_profile=True,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait if use_spot_instances else None,
    metric_definitions=[
       {'Name': 'train:step', 'Regex': 'step:(.*?);'},
       {'Name': 'train:loss', 'Regex': 'loss:(.*?);'},
       {'Name': 'train:avg-sts-score', 'Regex': 'Avg. STS:(.*?);'}
    ]
)

### Start Training job
S3에서 훈련 인스턴스로 복사될 데이터를 지정한 후 SageMaker 훈련 job을 시작합니다. 모델 크기, 데이터 세트 크기에 따라서 몇십 분에서 몇 시간까지 소요될 수 있습니다.

In [ ]:
fast_file = lambda x: TrainingInput(x, input_mode="FastFile")
estimator.fit(
    {
        "training": fast_file(s3_dataset_path),
    },
    wait=False
)

### View Training Job
SageMaker 콘솔 창에서 훈련 내역을 직접 확인할 수도 있지만, 아래 코드 셀에서 생성되는 링크를 클릭하면 더 편리하게 훈련 내역을 확인할 수 있습니다.

In [ ]:
from IPython.display import display, HTML

def make_console_link(region, train_job_name, train_task='[Training]'):
    train_job_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{train_job_name}">Training Job</a></b>'   
    cloudwatch_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={region}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={train_job_name};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a></b>'
    return train_job_link, cloudwatch_link  
        
train_job_name = estimator.latest_training_job.job_name
train_job_link, cloudwatch_link = make_console_link(region, train_job_name, '[Fine-tuning]')

display(HTML(train_job_link))
display(HTML(cloudwatch_link))

### Wait for the training jobs to complete
훈련이 완료될 때까지 기다립니다. estimator.fit(...)에서 wait=False로 설정한 경우, 아래 코드 셀의 주석을 해제 후 실행하여 동기 방식으로 변경할 수도 있습니다. 훈련 완료까지는 수십 분이 소요됩니다.

In [ ]:
sess.logs_for_job(job_name=train_job_name, wait=True)

<br>

## 3. Inference
---
#### Copy S3 model artifact to local directory
S3에 저장된 모델 아티팩트를 로컬 경로로 복사하여 압축을 해제합니다. 필요 시 로컬 환경에서 모델을 로드하여 추론을 수행할 수 있습니다.

In [ ]:
import json, os

local_model_dir = 'model_from_sagemaker'

if not os.path.exists(local_model_dir):
    os.makedirs(local_model_dir)

!aws s3 cp {estimator.model_data} {local_model_dir}/model.tar.gz
!tar -xzf {local_model_dir}/model.tar.gz -C {local_model_dir}
!rm -rf {local_model_dir}/model.tar.gz

### Load model artifact 

In [ ]:
import glob
import torch
import transformers
import numpy as np
from collections import OrderedDict
from transformers import BertForSequenceClassification, AutoTokenizer
from src.simcse import SimCSEModel
from src.infer import show_embedding_score

with open(f'{local_model_dir}/config.json') as f:
    json_object = json.load(f)
    
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')        
base_model = json_object["base_model"]
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = SimCSEModel(base_model)

In [ ]:
model_filename = glob.glob(f'{local_model_dir}/*.pt')[0]
state_dict = torch.load(model_filename)['model']

new_state_dict = {}
for key in state_dict:
    new_key = key.replace('module.','')
    new_state_dict[new_key] = state_dict[key]

model.load_state_dict(new_state_dict)
model = model.eval()

### Inference

In [ ]:
sentences = ['이번 주 일요일에 분당 이마트 점은 문을 여나요?',
             '일요일에 분당 이마트는 문 열어요?',
             '분당 이마트 점은 토요일에 몇 시까지 하나요']
show_embedding_score(tokenizer, model.cpu(), sentences)

### Evaluation

In [ ]:
from typing import List
from src.sts import STSEvaluation
from more_itertools import chunked
batch_size = 128
model = model.to(device)

@torch.inference_mode()
def encode(texts: List[str]) -> torch.Tensor:
    embs = []
    for text in chunked(texts, batch_size):
        batch: BatchEncoding = tokenizer(
            text,
            padding=True,
            truncation=True,
            return_tensors="pt",
        )
        emb = model(**batch.to(device))#, use_mlp=False)
        embs.append(emb.cpu())
    return torch.cat(embs, dim=0)

evaluation = STSEvaluation()
sts_metrics = evaluation(encode=encode)

In [ ]:
display(sts_metrics)